In [ ]:
import os
import numpy as np
import pandas as pd
from pathlib import Path

CSV_FILE = 'sdss_data.csv'
IMG_DIR = 'sdss_images'
IMG_SIZE = 128

df = pd.read_csv(CSV_FILE, comment = '#')

def load_img(df: pd.DataFrame, image_dir: Path) -> None:
    images = []
    valid_indices = []

    for index, row in df.iterrows():
        img_path = os.path.join(image_dir, f'image_{index:04d}.jpg')
        if os.path.exists(img_path):
            ...